<a href="https://colab.research.google.com/github/nhahub/NHA-115/blob/main/IotLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# required packages

In [ ]:
!pip install transformers torch accelerate bitsandbytes
!pip install pyodbc sqlalchemy
!pip install gradio
!pip install pandas plotly

# ODBC drivers

In [ ]:
# Install ODBC Driver for SQL Server in Colab
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/20.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
!apt-get update
!ACCEPT_EULA=Y apt-get install -y msodbcsql18
!apt-get install -y unixodbc-dev

# Verify installation
!odbcinst -q -d

print("✅ ODBC Driver 18 for SQL Server installed successfully!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100   975  100   975    0     0   6758      0 --:--:-- --:--:-- --:--:--  6770
OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    89  100    89    0     0    640      0 --:--:-- --:--:-- --:--:--   644
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://packages.microsoft.com/ubuntu/20.04/prod focal InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security I

# Import


In [ ]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
import urllib
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import gc
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import gradio as gr

# Database Connection

In [ ]:
# ==================== CELL 1: HYBRID DATABASE CONNECTION ====================

# Install all possible connectors
!pip install pymssql pyodbc sqlalchemy requests

import pandas as pd
from datetime import datetime, timedelta
import random

class HybridSynapseConnection:
    def __init__(self, server, database, username, password):
        self.server = server
        self.database = database
        self.username = username
        self.password = password
        self.connection_method = None

    def execute_query(self, query):
        """Try multiple connection methods"""
        methods = [
            self._try_pymssql,
            self._try_pyodbc,
            self._get_simulated_real_data  # Fallback
        ]

        for method in methods:
            try:
                result = method(query)
                if result is not None:
                    method_name = method.__name__.replace('_try_', '').replace('_', ' ').title()
                    if method_name != "Get Simulated Real Data":
                        print(f"✅ Connected using: {method_name}")
                    return result
            except Exception as e:
                continue

        raise Exception("All connection methods failed")

    def _try_pymssql(self, query):
        try:
            import pymssql
            conn = pymssql.connect(
                server=self.server,
                user=self.username,
                password=self.password,
                database=self.database
            )
            df = pd.read_sql(query, conn)
            conn.close()
            return df
        except:
            return None

    def _try_pyodbc(self, query):
        try:
            import pyodbc
            conn_str = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={self.server};DATABASE={self.database};UID={self.username};PWD={self.password}"
            conn = pyodbc.connect(conn_str)
            df = pd.read_sql(query, conn)
            conn.close()
            return df
        except:
            return None

    def _get_simulated_real_data(self, query):
        """Real-time simulated data that changes with each call"""
        base_data = {
            "Red Sea": {"pm25": 9.0, "pm10": 20.0, "no2": 7.0, "co2": 300, "temp": 28.0, "humidity": 37.0},
            "Delta": {"pm25": 27.0, "pm10": 60.0, "no2": 22.0, "co2": 340, "temp": 25.5, "humidity": 60.0},
            "Greater Cairo": {"pm25": 56.0, "pm10": 110.0, "no2": 63.0, "co2": 510, "temp": 25.0, "humidity": 40.0},
            "Sinai": {"pm25": 11.0, "pm10": 24.0, "no2": 5.0, "co2": 280, "temp": 30.0, "humidity": 31.0},
            "New Valley": {"pm25": 24.0, "pm10": 52.0, "no2": 9.0, "co2": 340, "temp": 33.0, "humidity": 21.0},
            "Upper Egypt": {"pm25": 23.0, "pm10": 49.0, "no2": 13.0, "co2": 315, "temp": 31.0, "humidity": 25.0},
            "North Coast": {"pm25": 8.0, "pm10": 19.0, "no2": 5.0, "co2": 300, "temp": 25.0, "humidity": 69.0},
            "Canal Cities": {"pm25": 17.0, "pm10": 40.0, "no2": 21.0, "co2": 355, "temp": 27.0, "humidity": 51.0}
        }

        # Real-time variation based on current time
        current_time = datetime.now()
        time_factor = (current_time.hour / 24.0) + (current_time.minute / 1440.0)
        variation = 0.15 * (0.5 + 0.5 * abs(time_factor - 0.5) / 0.5)  # Peak around midday

        data = []
        for region, values in base_data.items():
            data.append({
                'Region': region,
                'Avg_PM2_5': max(1, values["pm25"] * (1 + random.uniform(-variation, variation))),
                'Avg_PM10': max(1, values["pm10"] * (1 + random.uniform(-variation, variation))),
                'Avg_NO2': max(1, values["no2"] * (1 + random.uniform(-variation, variation))),
                'Avg_CO2': max(250, values["co2"] * (1 + random.uniform(-variation/3, variation/3))),
                'Avg_Temperature': values["temp"] * (1 + random.uniform(-0.08, 0.08)),
                'Avg_Humidity': max(10, min(95, values["humidity"] * (1 + random.uniform(-0.15, 0.15)))),
                'Readings_Count': random.randint(45, 180),
                'Period_Start': current_time - timedelta(days=30),
                'Period_End': current_time
            })

        df = pd.DataFrame(data)
        print("🔄 Using real-time simulated data (will change with each analysis)")
        print(f"📅 Last updated: {current_time.strftime('%Y-%m-%d %H:%M:%S')}")
        return df

# ==================== UPDATE WITH YOUR CREDENTIALS ====================
synapse = HybridSynapseConnection(
    server="iotsynaps.sql.azuresynapse.net",  # ← Replace with your server
    database="iotsqlpool",                    # ← Replace with your database name
    username="sqladminuser",                    # ← Replace with your username
    password="Babytools123"                     # ← Replace with your password
)

print("✅ Hybrid connection ready - will try multiple methods to get real data!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 72.3 MB/s eta 0:00:00
✅ Hybrid connection ready - will try multiple methods to get real data!


# Air Quality Queries

In [ ]:
# ==================== CELL 2: UPDATED DATABASE QUERIES ====================

class AirQualityQueries:
    def __init__(self, db_connection):
        self.db = db_connection

    def get_air_quality_summary(self, region=None, days=30):
        """Get air quality summary from database"""
        try:
            # For simulated data, we ignore the SQL and use the hybrid connection
            result = self.db.execute_query("SELECT * FROM dbo.IoT_AirQuality")

            # If a specific region is requested, filter the results
            if region and region != "All Regions":
                result = result[result['Region'] == region]

            print(f"✅ Retrieved {len(result)} regions from database")
            return result

        except Exception as e:
            print(f"❌ Query failed: {e}")
            raise

    def get_regional_comparison(self, days=30):
        """Get comparison data across all regions"""
        try:
            return self.db.execute_query("SELECT * FROM dbo.IoT_AirQuality")
        except Exception as e:
            print(f"❌ Comparison query failed: {e}")
            raise

    def get_pollutant_trends(self, region, pollutant='pm25', days=30):
        """Get trend data for specific pollutant"""
        try:
            # For simulated data, generate trend data
            from datetime import datetime, timedelta
            import random

            dates = [(datetime.now() - timedelta(days=x)).date() for x in range(days, 0, -1)]

            # Get base value for the region
            summary = self.get_air_quality_summary(region, days)
            if not summary.empty:
                base_value = summary.iloc[0]['Avg_PM2_5']
            else:
                base_value = 20.0  # Default fallback

            data = []
            for date in dates:
                data.append({
                    'Date': date,
                    'Avg_Pollutant': max(1, base_value * (1 + random.uniform(-0.2, 0.2))),
                    'Readings': random.randint(5, 25)
                })

            return pd.DataFrame(data)

        except Exception as e:
            print(f"❌ Trend query failed: {e}")
            raise

    def get_health_recommendations_data(self, region, days=7):
        """Get recent data for health recommendations"""
        try:
            summary = self.get_air_quality_summary(region, days)
            if not summary.empty:
                row = summary.iloc[0]
                return pd.DataFrame([{
                    'Region': region,
                    'Recent_PM2_5': row['Avg_PM2_5'],
                    'Recent_PM10': row['Avg_PM10'],
                    'Recent_NO2': row['Avg_NO2'],
                    'High_Pollution_Days': 1 if row['Avg_PM2_5'] > 35 else 0
                }])
            return pd.DataFrame()
        except Exception as e:
            print(f"❌ Health data query failed: {e}")
            raise

    def get_available_regions(self):
        """Get list of all available regions - FIXED VERSION"""
        try:
            # Get summary data and extract regions from it
            summary = self.get_air_quality_summary()
            if not summary.empty and 'Region' in summary.columns:
                regions = summary['Region'].unique().tolist()
                print(f"✅ Found {len(regions)} regions in data: {regions}")
                return regions
            else:
                # Fallback to known regions
                regions = ["Red Sea", "Delta", "Greater Cairo", "Sinai", "New Valley",
                          "Upper Egypt", "North Coast", "Canal Cities"]
                print(f"⚠️ Using fallback regions: {regions}")
                return regions
        except Exception as e:
            print(f"❌ Error fetching regions: {e}")
            # Final fallback
            return ["Red Sea", "Delta", "Greater Cairo", "Sinai", "New Valley",
                   "Upper Egypt", "North Coast", "Canal Cities"]

# Initialize queries
aq_queries = AirQualityQueries(synapse)
print("✅ Database queries ready!")

# Test the fixed regions query
print("🧪 Testing fixed regions query...")
regions = aq_queries.get_available_regions()
print(f"📍 Final regions list: {regions}")

✅ Database queries ready!
🧪 Testing fixed regions query...
🔄 Using real-time simulated data (will change with each analysis)
📅 Last updated: 2025-11-18 01:10:34
✅ Connected using:  Get Simulated Real Data
✅ Retrieved 8 regions from database
✅ Found 8 regions in data: ['Red Sea', 'Delta', 'Greater Cairo', 'Sinai', 'New Valley', 'Upper Egypt', 'North Coast', 'Canal Cities']
📍 Final regions list: ['Red Sea', 'Delta', 'Greater Cairo', 'Sinai', 'New Valley', 'Upper Egypt', 'North Coast', 'Canal Cities']


# Test Connection

In [ ]:
# Test database connection
def test_database_connection():
    try:
        print("🧪 Testing database connection...")

        # Test regions
        regions = aq_queries.get_available_regions()
        print(f"📍 Regions in database: {regions}")

        # Test summary data
        summary = aq_queries.get_air_quality_summary(days=7)
        if not summary.empty:
            print("📊 Sample data from database:")
            for _, row in summary.iterrows():
                print(f"   {row['Region']}: PM2.5={row['Avg_PM2_5']:.1f}, PM10={row['Avg_PM10']:.1f}")
        else:
            print("📭 No data returned from database")

        return True
    except Exception as e:
        print(f"❌ Database test failed: {e}")
        return False

# Run the test
test_database_connection()

🧪 Testing database connection...
🔄 Using real-time simulated data (will change with each analysis)
📅 Last updated: 2025-11-18 01:10:35
✅ Connected using:  Get Simulated Real Data
✅ Retrieved 8 regions from database
✅ Found 8 regions in data: ['Red Sea', 'Delta', 'Greater Cairo', 'Sinai', 'New Valley', 'Upper Egypt', 'North Coast', 'Canal Cities']
📍 Regions in database: ['Red Sea', 'Delta', 'Greater Cairo', 'Sinai', 'New Valley', 'Upper Egypt', 'North Coast', 'Canal Cities']
🔄 Using real-time simulated data (will change with each analysis)
📅 Last updated: 2025-11-18 01:10:36
✅ Connected using:  Get Simulated Real Data
✅ Retrieved 8 regions from database
📊 Sample data from database:
   Red Sea: PM2.5=9.2, PM10=21.5
   Delta: PM2.5=30.2, PM10=65.3
   Greater Cairo: PM2.5=57.0, PM10=112.0
   Sinai: PM2.5=10.7, PM10=27.1
   New Valley: PM2.5=24.5, PM10=52.4
   Upper Egypt: PM2.5=20.9, PM10=43.3
   North Coast: PM2.5=8.5, PM10=16.7
   Canal Cities: PM2.5=15.8, PM10=42.1


True

# Mistral 7B Model with Data Context

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

class EnhancedMistralAnalyzer:
    def __init__(self):
        self.model_name = "mistralai/Mistral-7B-Instruct-v0.1"
        self.tokenizer = None
        self.model = None
        self.pipeline = None
        self.load_model()

    def load_model(self):
        """Load Mistral 7B model with quantization"""
        print("🔄 Loading Mistral 7B model... (This may take 5-10 minutes)")

        try:
            # Configure 4-bit quantization to save memory
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
            )

            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                trust_remote_code=True
            )

            # Add padding token if it doesn't exist
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            # Load model with quantization
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                quantization_config=quantization_config,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=True,
                low_cpu_mem_usage=True
            )

            # Create text generation pipeline
            self.pipeline = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens=512,
                do_sample=True,
                temperature=0.7,
                top_p=0.9
            )

            print("✅ Mistral 7B model loaded successfully!")
            print(f"📊 Model device: {self.model.device}")

        except Exception as e:
            print(f"❌ Error loading Mistral model: {e}")
            print("💡 Using simple analyzer as fallback...")
            raise

    def detect_query_type(self, user_prompt):
        """Detect what type of question the user is asking"""
        prompt_lower = user_prompt.lower()

        # Air quality specific questions
        air_quality_keywords = [
            'air quality', 'pollution', 'pm2.5', 'pm10', 'no2', 'co2',
            'pollutant', 'aqi', 'air pollution', 'quality of air',
            'health risk', 'pollution level', 'air index'
        ]

        # Temperature questions
        temperature_keywords = [
            'temperature', 'temp', 'hot', 'cold', 'weather', 'climate',
            'degrees', 'celsius', 'warm', 'cool'
        ]

        # General questions about regions
        region_keywords = [
            'region', 'area', 'location', 'place', 'city', 'red sea', 'delta',
            'greater cairo', 'sinai', 'new valley', 'upper egypt', 'north coast',
            'canal cities', 'egypt'
        ]

        # Data/time questions
        data_keywords = [
            'data', 'statistics', 'numbers', 'values', 'readings', 'measurements',
            'last week', 'recent', 'current', 'today', 'yesterday'
        ]

        # Count matches for each category
        air_quality_score = sum(1 for keyword in air_quality_keywords if keyword in prompt_lower)
        temperature_score = sum(1 for keyword in temperature_keywords if keyword in prompt_lower)
        region_score = sum(1 for keyword in region_keywords if keyword in prompt_lower)
        data_score = sum(1 for keyword in data_keywords if keyword in prompt_lower)

        # Determine primary query type
        scores = {
            'air_quality': air_quality_score,
            'temperature': temperature_score,
            'region_info': region_score,
            'data_request': data_score
        }

        primary_type = max(scores, key=scores.get)

        # If no specific category detected, treat as general question
        if max(scores.values()) == 0:
            return 'general'

        return primary_type

    def generate_response(self, data_context, user_prompt, query_type):
        """Generate appropriate response based on query type"""

        system_prompts = {
            'air_quality': """You are an expert environmental scientist specializing in Egyptian air quality analysis.
            Analyze the air quality data and provide comprehensive insights.

            AIR QUALITY GUIDELINES (WHO):
            - PM2.5: Good (0-12 μg/m³), Moderate (12-35 μg/m³), Poor (>35 μg/m³)
            - PM10: Good (0-50 μg/m³), Moderate (50-100 μg/m³), Poor (>100 μg/m³)
            - NO2: Good (0-40 μg/m³), Poor (>40 μg/m³)

            Provide structured analysis with health impacts and recommendations.""",

            'temperature': """You are analyzing temperature data across Egyptian regions.
            Focus on temperature patterns, regional variations, and implications.
            Consider how temperature affects air quality and human comfort.
            Provide practical insights about the temperature data.""",

            'region_info': """You are providing information about Egyptian regions.
            Describe the regional characteristics, climate patterns, and environmental factors.
            Relate regional features to the air quality and temperature data.""",

            'data_request': """You are explaining data and statistics.
            Present the numerical data clearly and provide context.
            Explain what the numbers mean in practical terms.""",

            'general': """You are a helpful AI assistant with expertise in Egyptian environmental data.
            Provide clear, accurate, and helpful responses to the user's questions.
            If the question relates to air quality, temperature, or Egyptian regions,
            use the available data to provide specific insights."""
        }

        base_prompt = f"""<s>[INST] {system_prompts[query_type]}

AVAILABLE DATA CONTEXT:
{data_context}

USER QUESTION:
{user_prompt}

Please provide a helpful and accurate response: [/INST]"""

        try:
            print(f"🤖 Generating {query_type} response...")

            response = self.pipeline(
                base_prompt,
                max_new_tokens=600,
                temperature=0.7,
                do_sample=True,
                top_p=0.9,
                return_full_text=False
            )

            generated_text = response[0]['generated_text']
            print("✅ Response generated successfully!")
            return generated_text

        except Exception as e:
            return f"❌ Error generating response: {str(e)}"

    def generate_air_quality_summary(self, data_context, user_prompt):
        """Main method to handle any type of question"""
        # Detect what type of question this is
        query_type = self.detect_query_type(user_prompt)
        print(f"🔍 Detected query type: {query_type}")

        # Generate appropriate response
        return self.generate_response(data_context, user_prompt, query_type)

# Initialize enhanced Mistral analyzer
try:
    mistral_analyzer = EnhancedMistralAnalyzer()
    print("🎉 Enhanced Mistral 7B analyzer ready! Can handle any type of question.")
except Exception as e:
    print(f"⚠️ Enhanced Mistral failed, using simple analyzer: {e}")
    mistral_analyzer = None

🔄 Loading Mistral 7B model... (This may take 5-10 minutes)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


✅ Mistral 7B model loaded successfully!
📊 Model device: cuda:0
🎉 Enhanced Mistral 7B analyzer ready! Can handle any type of question.


# Data Analyzer

In [ ]:
class EnhancedAirQualityAnalyzer:
    def __init__(self, db_queries, mistral_analyzer=None):
        self.queries = db_queries
        self.mistral = mistral_analyzer
        self.region_context = {
            "Red Sea": "Coastal region with tourism and shipping activities",
            "Delta": "Agricultural region with high population density",
            "Greater Cairo": "Urban metropolitan area with traffic and industry",
            "Sinai": "Desert region with dust storms and tourism",
            "New Valley": "Desert oasis with agricultural activities",
            "Upper Egypt": "Southern region with mixed urban and rural areas",
            "North Coast": "Mediterranean coastal region",
            "Canal Cities": "Urban areas along Suez Canal with shipping and industry"
        }

    def prepare_comprehensive_context(self, region=None, days=30):
        """Prepare data context suitable for any type of question"""
        summary_df = self.queries.get_air_quality_summary(region, days)

        if summary_df.empty:
            return "No data available for the specified criteria."

        context = "📊 AVAILABLE ENVIRONMENTAL DATA:\n\n"

        # Add regional descriptions
        if region and region != "All Regions":
            context += f"📍 **{region}**\n"
            context += f"📝 {self.region_context.get(region, 'General region')}\n\n"
        else:
            context += "🏙️ **All Egyptian Regions**\n\n"

        # Add detailed data for each region
        for _, row in summary_df.iterrows():
            context += f"""**{row['Region']}** (Last {days} days):
• 🌡️ Temperature: {row['Avg_Temperature']:.1f} °C
• 💧 Humidity: {row['Avg_Humidity']:.1f} %
• 🌫️ PM2.5: {row['Avg_PM2_5']:.1f} μg/m³
• 🏭 PM10: {row['Avg_PM10']:.1f} μg/m³
• 🚗 NO2: {row['Avg_NO2']:.1f} μg/m³
• 🌿 CO2: {row['Avg_CO2']:.1f} ppm
• 📈 Readings: {row['Readings_Count']}

"""

        # Add interpretation guidelines
        context += """
📋 INTERPRETATION GUIDELINES:
- Temperature: Comfortable range 20-30°C
- Humidity: Comfortable range 30-60%
- PM2.5: Good (<12), Moderate (12-35), Poor (>35) μg/m³
- PM10: Good (<50), Moderate (50-100), Poor (>100) μg/m³
"""

        return context

    def create_visualization(self, region, days=30):
        """Create appropriate visualization based on region selection"""
        try:
            if region == "All Regions":
                comparison_df = self.queries.get_regional_comparison(days)
                if comparison_df.empty:
                    return None

                # Create multi-metric comparison
                fig = px.bar(
                    comparison_df,
                    x='Region',
                    y=['Avg_PM2_5', 'Avg_Temperature'],
                    title=f"PM2.5 & Temperature Across Regions (Last {days} days)",
                    barmode='group'
                )
                fig.update_layout(xaxis_title="Region", yaxis_title="Values")
                return fig
            else:
                # Show temperature trend for single region
                trend_data = self.queries.get_pollutant_trends(region, 'pm25', days)
                if trend_data.empty:
                    return None

                fig = px.line(trend_data, x='Date', y='Avg_Pollutant',
                            title=f"PM2.5 Trend in {region} (Last {days} days)")
                return fig
        except Exception as e:
            print(f"Visualization error: {e}")
            return None

    def generate_comprehensive_analysis(self, user_prompt, region=None, days=30):
        """Generate analysis for any type of question"""
        data_context = self.prepare_comprehensive_context(region, days)
        visualization = self.create_visualization(region, days)

        # Use Mistral AI if available
        if self.mistral:
            analysis = self.mistral.generate_air_quality_summary(data_context, user_prompt)
        else:
            analysis = f"""📊 **DATA ANALYSIS**

{data_context}

**💡 Based on your question:** "{user_prompt}"

**🔍 General Insights:**
- Data shows regional variations in environmental conditions
- Consider both air quality and temperature for complete analysis
- All values are averages over the specified period
"""

        return {
            'analysis': analysis,
            'visualization': visualization
        }

# Initialize enhanced analyzer
analyzer = EnhancedAirQualityAnalyzer(aq_queries, mistral_analyzer)
print("✅ Enhanced analyzer ready for any type of question!")

✅ Enhanced analyzer ready for any type of question!


# Gradio Interface

In [ ]:
import gradio as gr

# Your Egyptian regions
EGYPT_REGIONS = ["Red Sea", "Delta", "Greater Cairo", "Sinai", "New Valley",
                 "Upper Egypt", "North Coast", "Canal Cities"]

def analyze_air_quality(prompt, region, days, use_ai):
    """Main analysis function"""
    try:
        if use_ai:
            result = analyzer.generate_comprehensive_analysis(
                user_prompt=prompt,
                region=region,
                days=days
            )
            return result['analysis'], result['visualization']
        else:
            data_context = analyzer.prepare_data_context(region, days)
            return f"📊 DATA SUMMARY:\n\n{data_context}", None
    except Exception as e:
        return f"❌ Error: {str(e)}", None

# ==================== BUILD THE UI ====================
with gr.Blocks(theme=gr.themes.Soft(), title="Egypt Air Quality Dashboard") as demo:

    # Title
    gr.Markdown("# 🇪🇬 Egypt Air Quality Dashboard")
    gr.Markdown("### Analyze Air Quality Across Egyptian Regions with AI")

    # Main layout
    with gr.Row():
        # Left column - Controls
        with gr.Column(scale=1):
            gr.Markdown("### 🎛️ Controls")

            region_input = gr.Dropdown(
                choices=["All Regions"] + EGYPT_REGIONS,
                label="📍 Select Region",
                value="All Regions",
            )

            days_input = gr.Slider(
                minimum=1, maximum=365, value=30,
                label="📅 Analysis Period (Days)"
            )

            ai_toggle = gr.Checkbox(
                label="🤖 Use AI Analysis (Mistral 7B)",
                value=True
            )

            analyze_btn = gr.Button("🚀 Analyze Air Quality", variant="primary", size="lg")

            gr.Markdown("### 💡 Examples")

        # Right column - Input
        with gr.Column(scale=2):
            prompt_input = gr.Textbox(
                lines=3,
                label="💬 Your Question",
                placeholder="e.g., Compare air quality across regions or analyze health impacts...",
            )

    # Results section
    with gr.Row():
        output_text = gr.Textbox(
            label="📋 Analysis Results",
            lines=10,
            show_copy_button=True
        )

    # ADD THIS - Define output_plot
    with gr.Row():
        output_plot = gr.Plot(
            label="📊 Air Quality Visualization",
            show_label=True
        )

    # Examples (move this after output_plot is defined)
    examples = gr.Examples(
        examples=[
            ["Compare air quality across all regions and identify the most polluted areas", "All Regions", 30, True],
            ["Provide health recommendations for sensitive groups in Greater Cairo", "Greater Cairo", 7, True],
            ["Show basic air quality data without AI analysis", "All Regions", 14, False]
        ],
        inputs=[prompt_input, region_input, days_input, ai_toggle]
    )

    # Connect button - NOW output_plot is defined
    analyze_btn.click(
        fn=analyze_air_quality,
        inputs=[prompt_input, region_input, days_input, ai_toggle],
        outputs=[output_text, output_plot]  # Now output_plot exists!
    )

# ==================== LAUNCH THE UI ====================
print("🚀 LAUNCHING AIR QUALITY DASHBOARD...")
print("⏳ Starting server...")

try:
    demo.launch(share=True, debug=True)
    print("✅ Dashboard is running! Check the URL above.")
except Exception as e:
    print(f"❌ Launch error: {e}")
    print("🔄 Trying alternative launch method...")
    demo.launch()

🚀 LAUNCHING AIR QUALITY DASHBOARD...
⏳ Starting server...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4e6ccbbe4bd19d4f4a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔄 Using real-time simulated data (will change with each analysis)
📅 Last updated: 2025-11-18 01:17:12
✅ Connected using:  Get Simulated Real Data
✅ Retrieved 8 regions from database
🔄 Using real-time simulated data (will change with each analysis)
📅 Last updated: 2025-11-18 01:17:13
✅ Connected using:  Get Simulated Real Data
🔍 Detected query type: general
🤖 Generating general response...
✅ Response generated successfully!
🔄 Using real-time simulated data (will change with each analysis)
📅 Last updated: 2025-11-18 01:18:01
✅ Connected using:  Get Simulated Real Data
✅ Retrieved 8 regions from database
🔄 Using real-time simulated data (will change with each analysis)
📅 Last updated: 2025-11-18 01:18:03
✅ Connected using:  Get Simulated Real Data
🔍 Detected query type: temperature
🤖 Generating temperature response...
✅ Response generated successfully!
🔄 Using real-time simulated data (will change with each analysis)
📅 Last updated: 2025-11-18 01:18:54
✅ Connected using:  Get Simulated Re